# History
This is a script I made in response to accidentally importing a lot of calendar events form a .ics file to my personal calendar.
There's no way I can go in and remove all the added events by hand, since I don't know how far back they go. I also don't remember which event belonged to me vs. the other person.

This script calls the Google Calendar API and compares the the UID of the events found vs the UID in the .ics file, then removes the found events.

# Pre-reqs:
1. Install dependencies `pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client`
2. Create a service file and have that available
3. Share calender persmission with the service file account
4. Have the `.ics` file available

In [ ]:
# The replace this with the calendar you want to fix
calendar_id = 'fill_me_in' # e.g., 'primary'; could also be your personal email

# Verify Access

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Set up credentials and the API client
SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = 'path/to/your-service-account-file.json'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)

# Try to list events (if this succeeds, the service account has the correct permissions)
try:
    events_result = service.events().list(calendarId=calendar_id).execute()
    print("Service account has access to the calendar.")
    events = events_result.get('items', [])
    for event in events:
        print(event['summary'], event['start'].get('dateTime', event['start'].get('date')))
except Exception as e:
    print(f"Failed to access calendar: {e}")


# Load .ics and extract UIDs

In [ ]:
import pandas as pd

# Initialize an empty dictionary to store the event data
event_data = []

# Open and read the .ics file
with open('your_ics_file.ics', 'r') as file:
    current_event = {}
    for line in file:
        line = line.strip()
        
        # Check if the line starts a new event
        if line == "BEGIN:VEVENT":
            current_event = {}
        elif line == "END:VEVENT":
            event_data.append(current_event)
        else:
            # Split the line into FIELD and VALUE at the first colon or semicolon
            if ':' in line:
                field, value = line.split(':', 1)
            elif ';' in line:
                field, value = line.split(';', 1)
            else:
                continue  # Skip lines that don't match the expected format
            
            current_event[field] = value

# Convert the list of events into a DataFrame
df = pd.DataFrame(event_data)

In [ ]:
# items I want to delete
df['UID']

# Fetch your calendar events

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta

# Set up credentials and the API client
SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = 'path/to/your-service-account-file.json'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)


In [ ]:
# Define the time window
now = datetime.utcnow()
days_ago = 1050
t_start = (now - timedelta(days=days_ago))
t_end = t_start + timedelta(days=60)

# Call the Google Calendar API to list events created in the last hour
events_result = service.events().list(
    calendarId=calendar_id,
    timeMin=t_start.isoformat() + 'Z',
    timeMax=t_end.isoformat() + 'Z',
    singleEvents=True,
    orderBy='startTime'
).execute()
events = events_result.get('items', [])
print(f"{len(events)} events found")

In [ ]:
unwanted_events = [event for event in events if df['UID'].str.contains(event['iCalUID']).any()]
print(f"{len(unwanted_events)} unwanted events found")

In [ ]:
# Delete each event found
for event in unwanted_events:
    print(f"Deleting event: {event['summary']}")
    service.events().delete(calendarId=calendar_id, eventId=event['id']).execute()

print(f"Deleted {len(unwanted_events)} unwanted event(s)")